# Aethr - Active Learning / Feedback Fine-Tune\n\nRun with **Google Colab extension + T4 GPU**.\nThis notebook parses `data/feedback/` metadata and fine-tunes from `models/best.pt`\nafter you re-annotate uncertain samples into a YOLO dataset.

In [ ]:
from pathlib import Path\nimport os\nimport json\nimport pandas as pd\n\nPROJECT_ROOT = Path('/content/aethr')\nif PROJECT_ROOT.exists():\n    os.chdir(PROJECT_ROOT)\n\n!nvidia-smi\n!pip install -q -r requirements.txt

In [ ]:
feedback_dir = PROJECT_ROOT / 'data' / 'feedback'\nrecords = []\nfor json_file in sorted(feedback_dir.glob('*.json')):\n    payload = json.loads(json_file.read_text(encoding='utf-8'))\n    records.append({\n        'json_file': json_file.name,\n        'source_image': payload.get('source_image', ''),\n        'saved_image': payload.get('saved_image', ''),\n        'uncertain_count': payload.get('uncertain_count', 0),\n        'note': payload.get('note', '')\n    })\n\nfeedback_df = pd.DataFrame(records)\ndisplay(feedback_df.head())\nprint('Feedback items:', len(feedback_df))\nmanifest_path = feedback_dir / 'feedback_manifest.csv'\nfeedback_df.to_csv(manifest_path, index=False)\nprint('Saved manifest:', manifest_path)

## Re-annotation Step\n\nRe-annotate selected feedback images (or false positives/false negatives) in your label tool,\nthen export a YOLO dataset and place it under `data/raw/<feedback-dataset>/data.yaml`.

In [ ]:
from ultralytics import YOLO\n\nbase_model = PROJECT_ROOT / 'models' / 'best.pt'\nfeedback_data_yaml = PROJECT_ROOT / 'data' / 'raw' / 'solar-panel-defects-feedback' / 'data.yaml'\n\nif not feedback_data_yaml.exists():\n    raise FileNotFoundError(\n        f'Place corrected feedback dataset at: {feedback_data_yaml}'\n    )\n\nmodel = YOLO(str(base_model))\nresults = model.train(\n    data=str(feedback_data_yaml),\n    epochs=20,\n    imgsz=640,\n    batch=16,\n    project=str(PROJECT_ROOT / 'runs'),\n    name='aethr_feedback_finetune',\n    device=0\n)\nprint('Fine-tuning complete:', results.save_dir)